# Making the Most of your Colab Subscription



## Faster GPUs

Users who have purchased one of Colab's paid plans have access to faster GPUs and more memory. You can upgrade your notebook's GPU settings in `Runtime > Change runtime type` in the menu to select from several accelerator options, subject to availability.

The free of charge version of Colab grants access to Nvidia's T4 GPUs subject to quota restrictions and availability.

You can see what GPU you've been assigned at any time by executing the following cell. If the execution result of running the code cell below is "Not connected to a GPU", you can change the runtime by going to `Runtime > Change runtime type` in the menu to enable a GPU accelerator, and then re-execute the code cell.


In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Feb 11 10:29:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In order to use a GPU with your notebook, select the `Runtime > Change runtime type` menu, and then set the hardware accelerator to the desired option.

## More memory

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available. More powerful GPUs are always offered with high-memory VMs.



You can see how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is "Not using a high-RAM runtime", then you can enable a high-RAM runtime via `Runtime > Change runtime type` in the menu. Then select High-RAM in the Runtime shape toggle button. After, re-execute the code cell.


In [3]:
from transformers import pipeline

classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/ModernBERT-large-zeroshot-v2.0"
)

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/792M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.58M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Device set to use cuda:0


In [4]:
text = """About FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club. Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner. We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession. Community focused. Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia. We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams. We don't do normal. We overdeliver. From the way we teach, to the tools we provide, we believe in standing out and doing things differently. We believe in innovation and forward-thinking, and are constantly pushing the envelope with our resources and methods. We love good humans. We believe in treating people with respect and kindness. From strangers on the Internet, to our SRNAs and fellow colleagues, we practice the Golden Rule. We value transparency and doing the right thing, even when nobody is looking. We promise to bring inclusivity, openness and acceptance to every interaction. For the CRNA school applicant, helpful articles and resources. Check out our most popular applicant resource, The CRNA School Database. Applying to CRNA school? Our digital courses will take you further, faster. And that's why we founded The CRNA Club.    """
label_1 = "ad" # @param {"type":"string","placeholder":"ad"}
label_2 = "description" # @param {"type":"string","placeholder":"about us"}
labels = [].append(label_1)
labels.append(label_2)

import nltk
nltk.download("punkt")
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

# 4. Split text into segments (paragraphs or lines)
# Split into sentences or paragraphs:
segments = sent_tokenize(text)

print(f"Text split into {len(segments)} segments.")
# Prepare lists to store the two categories
spammy_segments = []
informative_segments = []

for seg in segments:
    # Clean up the segment (strip whitespace)
    seg = seg.strip()
    if not seg:
        continue  # skip empty lines

    # 5. Classify the segment using zero-shot classification
    result = classifier(seg, labels)

    print(f"Segment: {seg}")
    print(f"Classification result: {result['scores']}")


    # The pipeline's result["labels"] might reorder the labels by descending probability
    # so we find the index of each label in result["labels"]
    spammy_score = result["scores"][result["labels"].index(labels[0])]
    informative_score = result["scores"][result["labels"].index(labels[1])]

    # 6. Decide which category is higher
    if spammy_score > informative_score:
        spammy_segments.append(seg)
    else:
        informative_segments.append(seg)



AttributeError: 'NoneType' object has no attribute 'append'

In [ ]:
# 7. Print or otherwise use the results
print("===== SPAMMY SEGMENTS =====")
for s in spammy_segments:
    print(f"- {s}")

print("\n===== INFORMATIVE SEGMENTS =====")
for s in informative_segments:
    print(f"- {s}")

===== SPAMMY SEGMENTS =====
- About FLASH SALE!
- 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE!
- 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA.
- For the CRNA school applicant, helpful articles and resources.
- Applying to CRNA school?

===== INFORMATIVE SEGMENTS =====
- And that's why we founded The CRNA Club.
- Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner.
- We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession.
- Community focused.
- Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia.
- We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams.
- We don't do normal.
- We overdeliver.
- From the way we teach, to the tools we provide, we believe

In [ ]:
def classify_subchunk(sent_list):
    """
    Returns (best_label, best_score_log) for the entire subchunk.
    best_score_log = sum of log(probabilities) for the chosen label across all sentences.
    """
    # We'll sum up the log probabilities for each label across all sentences,
    # then pick which label has the higher total.
    total_ad_log = 0.0
    total_story_log = 0.0

    for s in sent_list:
        s = s.strip()
        if not s:
            continue
        result = classifier(s, labels)
        # e.g. result["labels"] = ["story", "advertisement"] in descending score order
        p_ad = result["scores"][result["labels"].index("advertisement")]
        p_story = result["scores"][result["labels"].index("story")]

        # add log probabilities
        total_ad_log += math.log(p_ad + 1e-9)
        total_story_log += math.log(p_story + 1e-9)

    # pick which label is best overall
    if total_ad_log > total_story_log:
        return ("advertisement", total_ad_log)
    else:
        return ("story", total_story_log)

# 4) Compare no-split vs. best single-split
# -- (a) No-split: treat entire chunk as one label
print(sentences)
a ="""25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club."""

whole_label, whole_score = classify_subchunk(a)

# -- (b) Single-split: try every boundary between sentences
best_split_score = float("-inf")
best_split = None  # store index of the best split
best_left_label = None
best_right_label = None

for w in range(1, len(sentences)):  # possible split points
    left = sentences[:w]
    right = sentences[w:]

    left_label, left_score = classify_subchunk(left)
    right_label, right_score = classify_subchunk(right)

    total_score = left_score + right_score

    if total_score > best_split_score:
        best_split_score = total_score
        best_split = w
        best_left_label = left_label
        best_right_label = right_label

print(f"No-split label: {whole_label}, log-score = {whole_score:.3f}")
if best_split:
    print(f"Best single-split at sentence {best_split}:")
    print(f" Left label: {best_left_label}")
    print(f" Right label: {best_right_label}")
    print(f" Total two-chunk log-score = {best_split_score:.3f}")

# 5) Decide final chunking
if best_split_score > whole_score:
    # We do the split
    final_left = sentences[:best_split]
    final_right = sentences[best_split:]

    print("\nSPLIT CHOSEN:")
    print("LEFT SUB-CHUNK (labeled as", best_left_label, "):")
    for s in final_left:
        print(" -", s)
    print("\nRIGHT SUB-CHUNK (labeled as", best_right_label, "):")
    for s in final_right:
        print(" -", s)
else:
    # Keep chunk as one
    print("\nNO SPLIT CHOSEN. The entire chunk is labeled:", whole_label)
    for s in sentences:
        print(" -", s)

NameError: name 'sentences' is not defined

# The DP approach

In [ ]:
#classifier = pipeline("zero-shot-classification", model="microsoft/deberta-large-mnli")


config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
import math
import nltk
from nltk.tokenize import sent_tokenize
from transformers import pipeline

# 1) DOWNLOAD NLTK TOKENIZER
nltk.download("punkt")

# 2) SET UP ZERO-SHOT CLASSIFIER
#classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 3) YOUR TEXT
text = """About FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE! 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA. And that's why we founded The CRNA Club. Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner. We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession. Community focused. Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia. We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams. We don't do normal. We overdeliver. From the way we teach, to the tools we provide, we believe in standing out and doing things differently. We believe in innovation and forward-thinking, and are constantly pushing the envelope with our resources and methods. We love good humans. We believe in treating people with respect and kindness. From strangers on the Internet, to our SRNAs and fellow colleagues, we practice the Golden Rule. We value transparency and doing the right thing, even when nobody is looking. We promise to bring inclusivity, openness and acceptance to every interaction. For the CRNA school applicant, helpful articles and resources. Check out our most popular applicant resource, The CRNA School Database. Applying to CRNA school? Our digital courses will take you further, faster. And that's why we founded The CRNA Club.    """


# CHOOSE YOUR LABELS
labels = ["advertisement", "introduction"]
# (Feel free to try synonyms like "promotional" vs. "educational")
def segment_text_viterbi(text, labels, classifier):
    import math
    import nltk
    from nltk.tokenize import sent_tokenize

    nltk.download("punkt", quiet=True)

    # 1) Tokenize into segments
    segments = sent_tokenize(text)

    # 2) Compute emission probabilities
    probs = []
    for seg in segments:
        seg = seg.strip()
        if not seg:
            probs.append((0.0, 0.0))
            continue
        result = classifier(seg, labels)
        p_ad = result["scores"][result["labels"].index(labels[0])]
        p_st = result["scores"][result["labels"].index(labels[1])]
        probs.append((p_ad, p_st))

    # 3) Define transition function
    def transition_cost(prev_label, curr_label):
        return 0.0 if prev_label == curr_label else -0.5

    # Convert to log probabilities
    import math
    log_probs = [(math.log(ad + 1e-9), math.log(st + 1e-9)) for (ad, st) in probs]
    n = len(segments)

    # 4) Initialize DP arrays
    dp = [[float('-inf')] * 2 for _ in range(n)]
    bp = [[None] * 2 for _ in range(n)]
    dp[0][0], dp[0][1] = log_probs[0][0], log_probs[0][1]

    # 5) Viterbi fill
    for i in range(1, n):
        for curr_label in (0, 1):
            for prev_label in (0, 1):
                score = (dp[i-1][prev_label]
                         + transition_cost(prev_label, curr_label)
                         + log_probs[i][curr_label])
                if score > dp[i][curr_label]:
                    dp[i][curr_label] = score
                    bp[i][curr_label] = prev_label

    # 6) Backtrack
    final_label = 0 if dp[n-1][0] > dp[n-1][1] else 1
    best_path = [final_label]
    for i in range(n-1, 0, -1):
        best_path.append(bp[i][best_path[-1]])
    best_path.reverse()

    # 7) Return segments paired with their labels
    label_map = {0: labels[0], 1: labels[1]}
    labeled_segments = [(segments[i].strip(), label_map[best_path[i]]) for i in range(n)]

    return labeled_segments



def print_segmented_text(labeled_segments):

    ads = [seg for seg, lbl in labeled_segments if lbl == labels[0]]
    stories = [seg for seg, lbl in labeled_segments if lbl == labels[1] ]

    print("===== ADVERTISEMENT SEGMENTS =====")
    for seg in ads:
        print("-", seg)

    print("\n===== STORY SEGMENTS =====")
    for seg in stories:
        print("-", seg)

def get_switching_strings(labeled_segments):

    best_path = [seg[1] for seg in labeled_segments]
    segments = [seg[0] for seg in labeled_segments]
    """
    When best_path[i] != best_path[i+1], combine segments[i] + " " + segments[i+1].
    Returns a list of these combined strings.
    """
    switching_strings = []
    switching_labels = []
    switching_indices = []
    n = len(segments)
    if n <= 1:
        return switching_strings

    for i in range(n - 1):
        if best_path[i] != best_path[i+1]:
            combined = segments[i] + " " + segments[i+1]
            #labels
            left_label = best_path[i]
            right_label = best_path[i+1]

            switching_strings.append(combined)
            switching_labels.append((left_label, right_label))
            switching_indices.append(i)

    return switching_strings, switching_labels,switching_indices

def score_chunk_for_label(text, classifier, labels, fixed_label):
    """
    Return log(prob) that 'text' belongs to 'fixed_label'.
    'labels' is the full set (e.g. ["advertisement", "story"]).
    """
    result = classifier(text, labels)
    prob = result["scores"][result["labels"].index(fixed_label)]
    return math.log(prob + 1e-9)

def best_split(chunk, left_label, right_label, classifier, all_labels):
    """
    Split 'chunk' at each possible word boundary.
    The left subchunk is forced to 'left_label';
    the right subchunk is forced to 'right_label'.
    Return (split_index, total_log_prob, best_left_str, best_right_str).
    """
    words = chunk.split()
    if len(words) < 2:
        # No meaningful split. Just compute log(prob) under left_label by default
        # or whichever label you'd like to enforce.
        logp = score_chunk_for_label(chunk, classifier, all_labels, left_label)
        return (None, logp, chunk, "")

    best_score = float("-inf")
    best_split_idx = None
    best_left_str = None
    best_right_str = None

    for i in range(1, len(words)):
        left_str = " ".join(words[:i])
        right_str = " ".join(words[i:])

        left_log = score_chunk_for_label(left_str, classifier, all_labels, left_label)
        right_log = score_chunk_for_label(right_str, classifier, all_labels, right_label)
        total_log = left_log + right_log

        if total_log > best_score:
            best_score = total_log
            best_split_idx = i
            best_left_str = left_str
            best_right_str = right_str

    return best_split_idx, best_score

### all together

labeled_segments = segment_text_viterbi(text, labels, classifier)
print_segmented_text(labeled_segments)
switching_strings,switching_labels, switching_indices = get_switching_strings(labeled_segments)

for i, s in enumerate(switching_strings):
    left_label, right_label = switching_labels[i]
    split_idx, score = best_split(s, left_label, right_label, classifier, labels)

    # Tokenize the chunk
    words = s.split()
    # Split at 'split_idx' in the word list
    left_str = " ".join(words[:split_idx])
    right_str = " ".join(words[split_idx:])
    print("\n\n corrected split:")
    print(f"{left_label}: {left_str}")
    print(f"{right_label}: {right_str}")
    print("Score:", score)
    print("-----")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


===== ADVERTISEMENT SEGMENTS =====
- About FLASH SALE!
- 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE!
- 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA.

===== STORY SEGMENTS =====
- And that's why we founded The CRNA Club.
- Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner.
- We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession.
- Community focused.
- Our goal is to foster a community of like-minded individuals who share the common passion of anesthesia.
- We love education, research and innovation, and our goal is to inspire those around us to take one step closer toward achieving their goals and dreams.
- We don't do normal.
- We overdeliver.
- From the way we teach, to the tools we provide, we believe in standing out and doing things differently.
- We believe in innovation and forward-think

In [ ]:
segment_text_viterbi(text, labels, classifier)

s

In [ ]:

# 3) Create a dict to store split replacements
split_replacements = {}
print(switching_labels)
# 4) Fix each switching chunk
for i, s in enumerate(switching_strings):
    print(i)
    left_label, right_label = switching_labels[i]
    seg_index = switching_indices[i]   # which segment we need to replace

    split_idx, score = best_split(s, left_label, right_label, classifier, labels)
    words = s.split()

    left_str = " ".join(words[:split_idx])
    right_str = " ".join(words[split_idx:])

    print("\nCorrected split:")
    print(f"{left_label}: {left_str}")
    print(f"{right_label}: {right_str}")
    print("Score:", score)
    print("-----")

    # Store the replacements (two sub-chunks) for this segment index
    split_replacements[seg_index] = [
        (left_str, left_label),
        (right_str, right_label)
    ]

# 5) Rebuild final segmentation
corrected_segments = []
for i, (seg_text, seg_label) in enumerate(labeled_segments):
    if i in split_replacements:
        # Replace with the two corrected sub-chunks
        corrected_segments.extend(split_replacements[i])
    else:
        # Keep the original segment
        corrected_segments.append((seg_text, seg_label))

# 6) Print final, corrected segmentation
print("\n===== FINAL CORRECTED SEGMENTATION =====")
for text_seg, label_seg in corrected_segments:
    print(f"[{label_seg}] {text_seg}")

[('advertisement', 'introduction')]
0

Corrected split:
advertisement: 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA.
introduction: And that's why we founded The CRNA Club.
Score: -0.8513471554173069
-----

===== FINAL CORRECTED SEGMENTATION =====
[advertisement] About FLASH SALE!
[advertisement] 25% OFF NEW TOOLKITS - USE CODE SAVE25FLASH SALE!
[advertisement] 25% OFF NEW TOOLKITS - USE CODE SAVE25 We're pretty obsessed with all things CRNA.
[introduction] And that's why we founded The CRNA Club.
[introduction] And that's why we founded The CRNA Club.
[introduction] Here at the CRNA Club our mission is to provide education and resources for the aspiring CRNA, student, and current practitioner.
[introduction] We are a community of anesthesia-obsessed individuals who are committed to sharing education and tools to further the profession.
[introduction] Community focused.
[introduction] Our goal is to foster a community of like-minded individuals who s